In [66]:
import pandas as pd

from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.mapping import WebMap, MapServiceLayer, MapFeatureLayer
from arcgis.features import FeatureLayer, GeoAccessor
from arcgis.geometry import filters

gis = GIS('home')

### Flood Prone Areas
* https://www.fema.gov/flood-maps/national-flood-hazard-layer
    * https://hazards-fema.maps.arcgis.com/apps/webappviewer/index.html?id=8b0adb51996444d4879338b5529aa9cd
* https://www.coast.noaa.gov/arcgis/rest/services/FloodExposureMapper/CFEM_FEMAFloodZones/MapServer/1
* https://landscape11.arcgis.com/arcgis/rest/services/USA_Flood_Hazard_Areas/ImageServer
* https://dukeuniv.maps.arcgis.com/home/item.html?id=3a2a84ccaa824fb6a87087553bf25f92


In [4]:
#Get the web mapping application item FEMA provides
fema_web_map_app = gis.content.get('8b0adb51996444d4879338b5529aa9cd')
fema_web_map_app.type

'Web Mapping Application'

In [5]:
#Get the data associated with the item
fema_web_map_app_data = fema_web_map_app.get_data()
#From the dictionary, extract the map's item ID
fema_web_map_app_ID = fema_web_map_app_data['map']['itemId']
#Find the item associated with that item 
fema_web_map_item = gis.content.get(fema_web_map_app_ID)
fema_web_map_item.type

'Web Map'

In [23]:
#Create a web map from the web map item
fema_web_map = WebMap(fema_web_map_item)

#Extract its layers
fema_web_map_layers = fema_web_map.layers

#Show the layer titles
[layer['title'] for layer in fema_web_map_layers]

['Effective FIRM Panels',
 'NFHL',
 'Coastal Barrier Resources System Area (US FWS)']

→ The layer we want is the `NFHL` (National Flood Hazard Layer); it is a *Map Service Layer*

In [49]:
#Get the URL for the MapServiceLayer
the_layer = fema_web_map.layers[1]
the_layer_url = the_layer.url
print(the_layer_url)

https://hazards.fema.gov/arcgis/rest/services/FIRMette/NFHLREST_FIRMette/MapServer


Browse the URL and you see that the Flood Hazard Layer is the layer with the ID 20, so we add `/20` to the URL


In [97]:
the_service_url = the_layer_url + '/20'

In [98]:
#Create a MapServiceLayer object from the url
fema_ms_layer = MapServiceLayer(fema_web_map_layers[1]['url'])
fema_layers = fema_ms_layer.properties['layers']
len(fema_layers)

32

In [99]:
#Fetch the layer
nhfl_layer = FeatureLayer(the_service_url)
nhfl_layer

<FeatureLayer url:"https://hazards.fema.gov/arcgis/rest/services/FIRMette/NFHLREST_FIRMette/MapServer/20">

In [100]:
#Create a spatial filter for Cartaret County
county_lyr = gis.content.get('14c5450526a8430298b2fa74da12c2f4').layers[0]
county_sdf = county_lyr.query("STATE_NAME = 'North Carolina' AND NAME = 'Carteret County'").sdf
county_geom = county_sdf.iloc[0,-1]
county_filter = filters.intersects(county_geom)

In [101]:
#Spatially query the nhfl feature layers
nhfl_fset = nhfl_layer.query(geometry_filter=county_filter)
nhfl_fset

<FeatureSet> 525 features

In [109]:
#Convert to sdf
nhfl_sdf = nhfl_fset.sdf
nhfl_sdf.spatial.to_featureclass('./data/Cartaret_NHFL.shp')

'V:\\Sandbox\\ArcGIS-PythonAPI\\data\\Cartaret_NHFL.shp'

### Sea Level Rise
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_1ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_1ft/MapServer/0
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_2ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_2ft/MapServer/0
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_3ft/MapServer/1
* https://www.coast.noaa.gov/arcgis/rest/services/dc_slr/slr_3ft/MapServer/0
* https://coast.noaa.gov/slrdata/DEMs/NC/index.html

### Storm Surge
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category1_v3/MapServer?f=html&cacheKey=8c1eda5e290c30bd
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category2_v3/MapServer?f=html&cacheKey=bc29bd820e1e09aa
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category3_v3/MapServer?f=html&cacheKey=990f926f2812a1e0
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category4_v3/MapServer?f=html&cacheKey=ba281138bc28168f
* https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/Storm_Surge_HazardMaps_Category5_v3/MapServer?f=html&cacheKey=a407a7ec582a37b0

### Areas of low slope
* 

### Soil erodibility
* https://landscape11.arcgis.com/arcgis/rest/services/USA_Soils_Erodibility_Factor/ImageServer
* https://gdg.sc.egov.usda.gov/GDGHome_DirectDownLoad.aspx

### Impervious Surfaces
* https://coast.noaa.gov/arcgis/rest/services/Landcover/CCAP_High_Resolution_V2_Impervious/ImageServer
* https://coast.noaa.gov/arcgis/rest/services/Landcover/CCAP_High_Resolution_V2_Landcover/ImageServer
* https://coastalimagery.blob.core.windows.net/ccap-landcover/CCAP_bulk_download/High_Resolution_Land_Cover/Phase_1_Initial_Layers/index.html